<a href="https://colab.research.google.com/github/BonneyBoja/Bonney-s-Portfolio/blob/main/Natural_Language_Processing_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries
!pip install transformers torch sentencepiece

In [ ]:
from transformers import BertTokenizer,BertModel
import numpy as np
import torch
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Loading Bert Model +Tokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=BertModel.from_pretrained('bert-base-uncased')

In [ ]:
#New Sentences Pairs
data= {
  "s1":  [
      "I left my phone at home today.",
        "The dog barked loudly at night.",
        "She is studying for her exam.",
        "The weather is very cold this morning.",
        "He bought a new laptop yesterday.",
        "The movie was interesting and fun.",
        "My brother works in a bank.",
        "The restaurant serves delicious food.",
        "The train arrived late again.",
        "She is painting a portrait in her room."
    ],
 "s2": [
    "I forgot my phone at the house today.",
        "The cat slept quietly on the sofa.",
        "She is preparing for her test.",
        "It is extremely chilly today.",
        "He purchased a brand new computer yesterday.",
        "The film was boring and too long.",
        "My sister works in a hospital.",
        "The café offers very tasty meals.",
        "The bus came early today.",
        "She is drawing a picture on the wall."
  ],
   # Ground truth labels (1 = similar, 0 = not similar)
    "truth": [
        1,  # phone left vs forgot phone (similar)
        0,  # dog barking vs cat sleeping (not similar)
        1,  # studying vs preparing for test (similar)
        1,  # cold weather vs chilly weather (similar)
        1,  # bought laptop vs bought computer (similar)
        0,  # interesting movie vs boring movie (opposites -> not similar)
        0,  # brother bank job vs sister hospital job (not similar)
        1,  # restaurant tasty food vs café tasty meals (similar)
        0,  # train late vs bus early (not similar)
        0   # painting portrait vs drawing on wall (not similar)
    ]
}
df=pd.DataFrame(data)

In [ ]:
#Extract CLS embeddings
def get_cls_embedding(sentence):
  inputs= tokenizer (sentence,return_tensors='pt',padding=True,truncation=True)
  with torch.no_grad():
    output = model(**inputs)
    return output.last_hidden_state[:,0,:].numpy()


In [ ]:
#compute cosine similarity
similarities=[]
predictions=[]
for i in range (len(df)):
  s1=df.iloc[i]['s1']
  s2=df.iloc[i]['s2']
  emb1=get_cls_embedding(s1)
  emb2=get_cls_embedding(s2)
  sim=cosine_similarity(emb1,emb2)[0][0]
  similarities.append(sim)
  pred=1 if sim >0.7 else 0
  predictions.append(pred)


In [ ]:
#Store result+accuracy
df["similarity"]=similarities
df['prediction']=predictions

accuracy = (df["truth"] == df["prediction"]).mean()

df, accuracy

(                                        s1  \
 0           I left my phone at home today.   
 1          The dog barked loudly at night.   
 2            She is studying for her exam.   
 3   The weather is very cold this morning.   
 4        He bought a new laptop yesterday.   
 5       The movie was interesting and fun.   
 6              My brother works in a bank.   
 7    The restaurant serves delicious food.   
 8            The train arrived late again.   
 9  She is painting a portrait in her room.   
 
                                              s2  truth  similarity  prediction  
 0         I forgot my phone at the house today.      1    0.983950           1  
 1            The cat slept quietly on the sofa.      0    0.908684           1  
 2                She is preparing for her test.      1    0.943727           1  
 3                 It is extremely chilly today.      1    0.954636           1  
 4  He purchased a brand new computer yesterday.      1    0.981334    